<a href="https://colab.research.google.com/github/AmelNozieres/AmelNozieres/blob/main/Fine_Tune_GPT_2_to_Write_Like_Ed_Sheeran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-Tune GPT-2 to Write Like Ed Sheeran**

# Generate Ed Sheeran Lyrics with GPT-2 🎤
>

In this notebook, we’ll use a pre-trained GPT-2 language model to generate lyrics in the style of Ed Sheeran.
We'll start by cleaning a dataset of lyrics, prompt the model with opening lines, and generate new verses.

There are two modes explored:
- Prompt-based generation using GPT-2 (zero-shot)
- Optional fine-tuning on Ed's lyrics for style consistency

Let’s write some heartbreak bangers in Python 😎


In [1]:
!pip install transformers datasets --quiet

from transformers import pipeline, set_seed, GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [2]:
with open("Ed_sheeran_songs.txt", "r", encoding="utf-8") as f:
    lyrics = f.read()
print(lyrics[:500])


"Castle On The Hill"
When I was six years old I broke my leg
I was running from my brother and his friends
And tasted the sweet perfume of the mountain grass I rolled down
I was younger then
Take me back to when
I found my heart and broke it here
Made friends and lost them through the years
And I've not seen the roaring fields in so long
I know I've grown
But I can't wait to go home
I'm on my way
Driving at 90 down those country lanes
Singing to "Tiny Dancer"
And I miss the way
You make me feel



In [8]:
import re

def clean_lyrics(text):
    text = re.sub(r'\[.*?\]', '', text)  # remove [Verse], [Chorus], etc.
    text = re.sub(r'\n+', '\n', text)    # remove extra newlines
    text = text.strip()
    return text

cleaned_lyrics = clean_lyrics(lyrics)


In [10]:
with open("Ed_sheeran_songs.txt", "r", encoding="utf-8") as f:
    corpus = f.read()

In [5]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")
set_seed(42)

prompt = "When I was younger I saw"
output = generator(prompt, max_length=100, num_return_sequences=1)

print(output[0]["generated_text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


When I was younger I saw my father growing mad. He would beat up my sister for having my father-in-law's back. I was so frustrated."

Her mother says she got a call telling her this was a boy she believed was in prison while she was younger, and she knew he was too young.

"They brought me out there in the dark at midnight to meet Mr. O'Brien," she says.

What happened next prompted both of the siblings


In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have pad token by default


In [11]:
from datasets import Dataset
import torch

# Split text into chunks of ~512 tokens
lines = corpus.split("\n\n")
examples = tokenizer(lines, truncation=True, padding=True)

dataset = Dataset.from_dict({
    "input_ids": examples["input_ids"],
    "attention_mask": examples["attention_mask"]
})


In [12]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [16]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# ✅ Disable Weights & Biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./results-edlyrics",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_steps=200,
    save_total_limit=1,
    logging_steps=50,
    prediction_loss_only=True
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [17]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.6299384435017905, metrics={'train_runtime': 99.5615, 'train_samples_per_second': 0.03, 'train_steps_per_second': 0.03, 'total_flos': 1567752192000.0, 'train_loss': 2.6299384435017905, 'epoch': 3.0})

In [25]:
from transformers import pipeline

# Reload the model
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "I found love in the pouring rain and now I’m stuck with all this pain"
result = generator(prompt, max_length=300, num_return_sequences=1)

print(result[0]["generated_text"])


Device set to use cpu


I found love in the pouring rain and now I’m stuck with all this pain… I am here with you…

And I am not going to let you stop me.

And I don't even need to go out so I’s not even getting drunk in the beginning. But now your pain is taking place in me.

This time… oh please. It seems it is just not me.

I'm going to stay here for you.
As long as I keep going alone on my back. I'm not even going to let you come with me.
I'm not even going to get drunk.

I'm not going to touch you.
I'm going to give you every night.

And I'm not going to let you let me down.

I'm not going to let you stop me.

Please.

What are you going to do to me?

I didn't know you could touch me.
I mean…

If you let me go alone?
If you let me go alone?

I'm not going to let you do it, do you need to?

But now I am not going to let you leave me alone…

Or give me a chance.
Or let me go alone and take you right now.

But I know you will not let go alone.

I'm going to give you a chance
